# Autodetect Home / Office network + Proxy
> "Depending on my location (home, office, hotspot), I want my network configuration to adapt and set the proper settings: proxy for http, conda, git, etc"
- show_tags: true
- toc: true
- branch: master
- badges: false
- comments: true
- categories: [WSL, linux, bash]

## IP detection

Command to get IP address is as follow:

```bash
IP=`ifconfig | grep 'inet '| grep -v '127.0.0.1' | cut -d: -f2 | awk '{ print $2}'`
```

I can then check how IP is setup:
- empty: no network attached, in that case nothing to do
- HOME_IP=192.168.1.241: based on MAC I give fixed IP to my computers (out of DHCP scope)
- S8_IP=192.168.: hotspot from samsung is using 192.168. addresses
- OFFICE_IP=10.: office network uses 10. addresses



Detect if variable `IP` is set:

```bash
if [ -z "$IP" ]; then
        echo "Not connected to any network"
fi
```

## Network detection and proxy settings


Depending on my network, I have to set or unset proxy.

Here is the 1st version:

```bash
(xgboost) guillaume@LL11LPC0PQARQ:~$ cat my_ip.sh
#!/bin/bash

IP=`ifconfig | grep 'inet '| grep -v '127.0.0.1' | cut -d: -f2 | awk '{ print $2}'`
HOME_IP=192.168.1.241
OFFICE_IP=10.
S8_IP=192.168.

# Set Proxy
function setproxy() {
     echo "Calling setproxy"
     export {http,https,ftp}_proxy="http://proxy_ip:80"
     export {HTTP,HTTPS,FTP}_PROXY="http://proxy_ip:80"
}

# Unset Proxy
function unsetproxy() {
     echo "Calling unsetproxy"
     unset {http,https,ftp}_proxy
     unset {HTTP,HTTPS,FTP}_PROXY
}

if [ -z "$IP" ]; then
        echo "Not connected to any network"
else
        echo "Connected and IP address is: $IP"

        if [[ "$IP" == "$HOME_IP" ]]; then
                echo "Connected at home from freebox pop --> no proxy"
                unsetproxy
        else
                if [[ "$IP" == "$S8_IP"* ]]; then
                     echo "Connected with mobile phone --> no proxy"
                     unsetproxy
                fi
                if [[ "$IP" == "$OFFICE_IP"* ]]; then
                        echo "Connected from Office --> proxy"
                        setproxy
                fi
        fi
fi
```


## Call this script: source

If I want these environment variables to be available from parent shell, I have to call my script with source.

```bash
(xgboost) guillaume@LL11LPC0PQARQ:~$ source my_ip.sh
Connected and IP address is: 10.xxx.xxx.xxx
192.168.1.241
Connected from Office --> proxy
Calling setproxy
```

And I will auto launch this script each time I open a terminal by adding `source my_ip.sh` at the end of `.bashrc`

## git and keep dot configuration files: config

Another great practice from Jeremy Howard: 
From https://developer.atlassian.com/blog/2016/02/best-way-to-store-dotfiles-git-bare-repo/
and
https://www.atlassian.com/git/tutorials/dotfiles



I will create a blog entry about that later.

```
config add .bashrc my_ip.sh
config commit -m 'detect network and set proxy'
config push
```


## wget: proxy / no proxy

I store proxy conf files under `~/proxy_files/`

For wget: 2 files
```bash
$ cat proxy_files/.wgetrc_noproxy
use_proxy=no
$ cat proxy_files/.wgetrc_proxy
use_proxy=yes
http_proxy=proxy_ip:80
https_proxy=proxy_ip:80
```


And enabling proxy for wget: `ln -sf ~/proxy_files/.wgetrc_proxy ~/.wgetrc`

Disabling proxy for wget: `ln -sf ~/proxy_files/.wgetrc_noproxy ~/.wgetrc`

So the updated functions `setproxy` and `unsetproxy` are:

```bash
# Set Proxy
function setproxy() {
     echo "Calling setproxy"
     export {http,https,ftp}_proxy="http://proxy_ip:80"
     export {HTTP,HTTPS,FTP}_PROXY="http://proxy_ip:80"
     #proxy for wget
     ln -sf ~/proxy_files/.wgetrc_proxy ~/.wgetrc
}

# Unset Proxy
function unsetproxy() {
     echo "Calling unsetproxy"
     unset {http,https,ftp}_proxy
     unset {HTTP,HTTPS,FTP}_PROXY
     #no proxy for wget
     ln -sf ~/proxy_files/.wgetrc_noproxy ~/.wgetrc
}
```

## apt-get: proxy / no proxy

I store proxy conf files under `~/proxy_files/`

For apt, 1 file
```bash
$ cat proxy_files/apt_proxy.conf
Acquire {
  HTTP::proxy "http://proxy_ip:80";
  HTTPS::proxy "http://proxy_ip:80";
}
```

And enabling proxy for apt: `sudo ln -sf ~/proxy_files/apt_proxy.conf /etc/apt/apt.conf.d/proxy.conf`

Disabling proxy for wget: `sudo rm -f /etc/apt/apt.conf.d/proxy.conf`

{% include alert.html text="Refactor to avoid password request each time it is launched" %}

for the moment I have just commented out these lines

So the updated functions `setproxy` and `unsetproxy` are:

```bash
# Set Proxy
function setproxy() {
     echo "Calling setproxy"
     export {http,https,ftp}_proxy="http://proxy_ip:80"
     export {HTTP,HTTPS,FTP}_PROXY="http://proxy_ip:80"
     #proxy for wget
     ln -sf ~/proxy_files/.wgetrc_proxy ~/.wgetrc
     #proxy for apt
     #sudo ln -sf ~/proxy_files/apt_proxy.conf /etc/apt/apt.conf.d/proxy.conf
}

# Unset Proxy
function unsetproxy() {
     echo "Calling unsetproxy"
     unset {http,https,ftp}_proxy
     unset {HTTP,HTTPS,FTP}_PROXY
     #no proxy for wget
     ln -sf ~/proxy_files/.wgetrc_noproxy ~/.wgetrc
     #no proxy for apt
     #sudo rm -f /etc/apt/apt.conf.d/proxy.conf
}
```

## Sept-21 2020: IP detection to be changed after WSL2

With WSL2, IP address is from 172 network.

This looks like a virtual internal address. More detail at that address: https://github.com/microsoft/WSL/issues/4150.

{% include alert.html text="to update IP detection" %}

## Oct-21 2020: Use git with github (ssh) behind corporate proxy

Here is the new configuration explained in my blog entry [Use git with github (ssh) behind corporate proxy](/guillaume_blog/git/wsl/2020/10/21/github-ssh-behind-proxy.html)

It is just a matter of linking appropriate files when I am in or out of corporate network.

As in `my_ip.sh`:

```bash
# Set Proxy
function setproxy() {
     echo "Calling setproxy"
     export {http,https,ftp}_proxy="http://proxy_ip:80"
     export {HTTP,HTTPS,FTP}_PROXY="http://proxy_ip:80"
     #proxy for wget
     ln -sf ~/proxy_files/.wgetrc_proxy ~/.wgetrc
     #proxy for apt
     #sudo ln -sf ~/proxy_files/apt_proxy.conf /etc/apt/apt.conf.d/proxy.conf
     #proxy for conda
     ln -sf ~/proxy_files/.condarc_proxy ~/.condarc
     #proxy for git
     git config --global http.proxy http://proxy_ip:80
     ln -sf ~/proxy_files/ssh_config_proxy ~/.ssh/config
}

# Unset Proxy
function unsetproxy() {
     echo "Calling unsetproxy"
     unset {http,https,ftp}_proxy
     unset {HTTP,HTTPS,FTP}_PROXY
     #no proxy for wget
     ln -sf ~/proxy_files/.wgetrc_noproxy ~/.wgetrc
     #no proxy for apt
     #sudo rm -f /etc/apt/apt.conf.d/proxy.conf
     #no proxy for conda
     ln -sf ~/proxy_files/.condarc_noproxy ~/.condarc
     #no proxy for git
     git config --global --unset http.proxy
     ln -sf ~/proxy_files/ssh_config_noproxy ~/.ssh/config
}
```